# Fitting Parameters for Virus Model

Analysis of the virus model.

 We will fit each of these patients separately, obtaining different values of parameters.

**Fitting**

 There are 6 patients with one value to fit. So, there are 6 different fits.


 Influenza, SARS and SARS-CoV3 in Tellurium

## A simple target cell-limited model, T ==> E ==> I --> produce V:
 T - number of target cells
 E - number of exposed cells (virus replicating inside, not yet spreading virus)
 I - number of infected cells (active virus production)
 V - viral titre, in units of TCID50/ml of biofluid wash (for Influenza)

# The ODEs
 $\frac{dT}{dt} = - \beta T V$
 
 dE/dt =   beta*T*V - kappa*E;
 dI/dt =   kappa * E - delta*I;
 dV/dt =   p*y(I) - c*y(V);

 All viral data is in log10(load),...
 log10(load predicted by model) may be needed for data fitting


Influenza.csv
 Influenza A data - 5 patients
 viral levels in log10(TCID50 / ml of nasal wash)
 time in days since volunteer exposure
 each line in the array is an individual volunteer



SARS_CoV2_sputum.csv and SARS_CoV2_nasal.csv
 SARS-CoV-2 data - 9 patients,
 for each patient - viral loads from lungs (sputum) and from nasal cavity (swab)
 viral levels in log10(RNA copies / ml sputum), ...
 respectively log10(RNA copies / nasal swab)
 time in days since symptoms onset
 corresponding lines in the two arrays belong to an individual patient



SARS.csv
 SARS data recorded from 12 patients;
 included them just for comparison, probably too few datapoints for model inference
virus_analysis.py [python] format: unix; [1,1]                                                                                                                                   
viral levels in log10(RNA copies / ml of nasopharingeal aspirate)
 time - only three samples per patient, at 5, 10 and 15 days post symptoms onset
"""

In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SBstoat
from SBstoat.modelStudy import ModelStudy
%matplotlib inline

## Antimony Model

In [2]:
ANTIMONY_MODEL  = '''
    // Equations
    E1: T -> E ; beta*T*V ; // Target cells to exposed
    E2: E -> I ; kappa*E ;  // Exposed cells to infected
    E3: -> V ; p*I ;        // Virus production by infected cells
    E4: V -> ; c*V ;        // Virus clearance
    E5: I -> ; delta*I      // Death of infected cells    

    // Parameters - from the Influenza article,
        
    beta = 3.2e-5;  // rate of transition of target(T) to exposed(E) cells, in units of 1/[V] * 1/day
    kappa = 4.0;    // rate of transition from exposed(E) to infected(I) cells, in units of 1/day
    delta = 5.2;    // rate of death of infected cells(I), in units of 1/day
    p = 4.6e-2;     // rate virus(V) producion by infected cells(I), in units of [V]/day
    c = 5.2;        // rate of virus clearance, in units of 1/day

    // Initial conditions
    T = 4E+8 // estimate of the total number of susceptible epithelial cells
             // in upper respiratory tract)
    E = 0
    I = 0
    V = 0.75 // the dose of virus in TCID50 in Influenza experiment; could be V=0 and I = 20 instead for a natural infection

'''

## Model Study
The following are the steps to study parameter fits for the patients.

### 1. Data Setup
Create separate timeseries for each patient.

In [3]:
"""
Transform the input data
"""

DIR = "/home/ubuntu/SBstoat/examples/virus"
INPUT_FILE =  os.path.join(DIR, "Influenza.csv")
NUM_BOOTSTRAP_ITERATIONS = 10000
VIRUS = "V"


# Convert input file to correct format
dataDF = pd.read_csv(INPUT_FILE, header=None)
dataDF = dataDF.transpose()
dataDF.index.name = "time"
patientDct = {p: "P%d" % (p+1) for p in range(6)}
dataDF = dataDF.rename(columns=patientDct)
observedTS = SBstoat.NamedTimeseries(dataframe=dataDF)

# Create separate data sources
dataSources = {}
for colname in observedTS.colnames:
    newTS = observedTS.subsetColumns([colname])
    newTS[VIRUS] = newTS[colname]
    newTS = newTS.subsetColumns([VIRUS])
    dataSources[colname] = newTS


In [4]:
# Here's what the data look like
for colname in observedTS.colnames:
    print("\n\n%s" % colname)
    print(dataSources[colname])



P1
        V
time     
0.0   2.0
1.0   5.5
2.0   4.0
3.0   5.5
4.0   3.0
5.0   0.0
6.0   0.0


P2
        V
time     
0.0   1.0
1.0   6.0
2.0   3.0
3.0   1.5
4.0   3.5
5.0   1.3
6.0   0.0


P3
        V
time     
0.0   2.5
1.0   5.0
2.0   5.0
3.0   3.0
4.0   5.5
5.0   3.5
6.0   0.0


P4
        V
time     
0.0   3.5
1.0   5.5
2.0   6.5
3.0   5.5
4.0   3.5
5.0   4.0
6.0   0.0


P5
        V
time     
0.0   2.5
1.0   3.0
2.0   6.5
3.0   6.5
4.0   2.0
5.0   0.8
6.0   0.0


P6
        V
time     
0.0   4.0
1.0   5.0
2.0   5.5
3.0   7.5
4.0   5.5
5.0   1.3
6.0   0.0


### 2. Transform the Simulation Results to Units of Observed Values
The observed values are in units of log10. So, simulation results must
be converted to these units. This is done by specifying a data transformation.
The data transformation is a function. It accepts as input the timeseries produced by the simulation;
the output is a numpy array.

In [5]:
def transformDataArr(timeseries):
    """
    Changes the timeseries to log units
    """
    arr = np.array([1 if v < 1 else v
          for v in timeseries[VIRUS]])
    return np.log10(arr)

### 3. Specify the name and range of permissible values for parameters
For each parameter, provide a tuple of its: lower bound, upper bound, and starting value.

In [6]:
# Parameter value ranges: lower, upper, initial value
parameterDct = dict(
      beta=(0, 10e-5, 3.2e-5),
      kappa=(0, 10, 4.0),
      delta=(0, 10, 5.2),
      p=(0, 1, 4.6e-2),
      c=(0, 10, 5.2)
      )

### 4. Run the model and produce plots.

In [9]:
dirStudyPath = os.path.join(DIR, "ModelStudyFitters")

# Study of model fitting
study = ModelStudy(ANTIMONY_MODEL, dataSources,
                  parameterDct=parameterDct,
                  dirStudyPath=dirStudyPath,
                  fittedDataTransformDct={"V": transformDataArr},
                  isSerialized=False)

study.bootstrap(numIteration=50)
study.plotFitAll()
study.plotParameterEstimates()




Doing bootstrapp for instance P1

**Running bootstrap for 5000 iterations with 4 processes.


Process Process-2:
Process Process-1:
Process Process-3:


Caught exception in main.


Process Process-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()


ValueError: Must provide a non-empty list

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/SBstoat/SBstoat/_modelFitterBootstrap.py", line 125, in _runBootstrap
    newFitter.fitModel(params=fitter.params)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/SBstoat/SBstoat/_modelFitterCore.py", line 376, in fitModel
    max_nfev=max_nfev)
  File "/home/ubuntu/SBstoat/SBstoat/_modelFitterBootstrap.py", line 125, in _runBootstrap
    newFitter.fitModel(params=fitter.params)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/SBstoat/SBst